# Data cleaning

In [21]:
import os
import pandas as pd
import numpy as np

# Load state dictionary for matching between two tables
state_dict = pd.read_csv('state_names.csv')

# Init variables
spending_dir = os.getcwd() + '/spending/FY18-19'
all_files = os.listdir(spending_dir)
states, med_spending, valid_rows_pct, states_abbrev = [], [], [], []

In [22]:
# Loop through each file for processing
for filename in all_files:
    if filename == '.DS_Store':
        continue
    file_path = os.path.join(spending_dir, filename)
    
    df = pd.read_excel(file_path,sheet_name='FY 2018-19')
        
    # Obtain state abbreviation and full name
    cur_state_abbrev = filename[0:2]
    states_abbrev.append(cur_state_abbrev)
    cur_state_full = state_dict['state'][state_dict['code']==cur_state_abbrev].iloc[0]
    states.append(cur_state_full)
    
    # Get some attributes
    og_len = df.shape[0] # original number of rows
    f33_sum = sum(df.flag_f33) # number of rows with flag
    
    # Remove rows with NAs or flags
    df = df.dropna(subset=['pp_total_norm_NERDS'])
    df = df[df.flag_f33!=1]
    df = df[df.flag_nerds!=1]
    new_len = df.shape[0]
    
    # Get median of all schools' spending
    med_spending.append(np.median(df['pp_total_norm_NERDS']))
    
    # Get percentage of valid rows out of total original rows
    valid_rows_pct.append(new_len/og_len)

In [43]:
# Create table of median per-state spending
spending_data = {'state': states,
                 'state_abbrev': states_abbrev,
                  'spending': med_spending,
                  'pct_valid': valid_rows_pct}
spending = pd.DataFrame(spending_data)
spending.round(2).head()

,state,state_abbrev,spending,pct_valid
0,Connecticut,CT,20765.79,0.90
1,Maine,ME,14692.88,0.91
2,Utah,UT,7842.16,0.86
3,Ohio,OH,12446.65,0.96
4,North Dakota,ND,14608.61,0.96


In [44]:
# Load test scores
scores_math = pd.read_csv(os.getcwd() + '/outcome/g8_math_2019.csv')
scores_math.rename(columns={'MN': 'math', 'Jurisdiction': 'state'}, inplace=True)

scores_reading = pd.read_csv(os.getcwd() + '/outcome/g8_reading_2019.csv')
scores_reading.rename(columns={'MN': 'reading', 'Jurisdiction': 'state'}, inplace=True)

scores_reading.head()

,state,reading,SigDiff,SigSymbol,AB,AP
0,DoDEA,280.28,18.28,▲,90.51,52.26
1,Massachusetts,273.11,11.11,▲,81.07,44.65
2,New Jersey,270.36,8.36,▲,76.51,42.87
3,Connecticut,269.72,7.72,▲,77.51,41.01
4,Vermont,268.48,6.48,▲,76.81,40.23


In [45]:
# Combine spending and test scores into one dataframe
all_data = pd.merge(spending, scores_math[['state','math']], on='state')
all_data = pd.merge(all_data, scores_reading[['state','reading']], on='state')
all_data.reading = all_data.reading.astype(float)

all_data.round(2).head()

# Save data
# data.to_csv('school_spending_outcome.csv', index=False)

,state,state_abbrev,spending,pct_valid,math,reading
0,Connecticut,CT,20765.79,0.90,286.16,269.72
1,Maine,ME,14692.88,0.91,282.37,265.17
2,Utah,UT,7842.16,0.86,284.93,267.42
3,Ohio,OH,12446.65,0.96,285.71,267.05
4,North Dakota,ND,14608.61,0.96,285.56,263.23
